In [1]:
"""
*** Instructions (assumes familiarity with Jupyter notebook) ***

1. To begin, create a dataset by calling prep(window), this concatenates, pre-processes and builds the features for all the collected accelerometer+gyroscope data
in the /data folder:

2. To run tests, it is useful to create test data. This is done with prep_test(file_to_test) - this function omits
setting the state, but the name tells you what the expected outcome should be (see the data README file for file naming
guidelines)

3. To test the model, run the test_model function. For highest accuracy, you should set the Polynomial degree to 3, though this will
run significantly slower

3. Next, we add the HMM for smoothing. Using domain specific knowledge, we setup an emission and transition
probability matrix, which captures the liklihood of all movement between the total number of states. 



Y. Optional, use the state_reconciler on data involving stand ups to readjust the predicted states. Particularly
aimed at fixing OCG / YMOUNT errors


def main(training_window=40):
    prepped_data = prep(training_window)



ALGORITHM STEPS:

1) The first (standing up detection) RF classifier prediction output sets the 'avg_stand' 
feature in the dataframe passed to (2)

2) The main (position state) RF classifier runs and outputs predicted states

3) The "reconcile function" looks through the predicted states and for OCG and YMOUNT looks for any stand up
motions detected in the vicinity. It corrects accordingly

4) The HMM smooths the results to create clean sequences


** In future, step 1 could be replicated with other isolated movements (e.g. bridge, shrimp, sit up)

TODO: is it possible to attempt to detect stand ups using a different time window?
"""

'\n*** Instructions (assumes familiarity with Jupyter notebook) ***\n\n1. To begin, create a dataset by calling prep(window), this concatenates, pre-processes and builds the features for all the collected accelerometer+gyroscope data\nin the /data folder:\n\n2. To run tests, it is useful to create test data. This is done with prep_test(file_to_test) - this function omits\nsetting the state, but the name tells you what the expected outcome should be (see the data README file for file naming\nguidelines)\n\n3.\n\n3. Next, we add the HMM for smoothing. Using domain specific knowledge, we setup an emission and transition\nprobability matrix, which captures the liklihood of all movement between the total number of states. \n\n\n\nY. Optional, use the state_reconciler on data involving stand ups to readjust the predicted states. Particularly\naimed at fixing OCG / YMOUNT errors\n\n\ndef main(training_window=40):\n    prepped_data = prep(training_window)\n\n\n\nALGORITHM STEPS:\n\n1) The firs

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

#from hmmlearn import hmm
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

from data_prep import combine_state_features, prep, prep_test
from rolltec_features import create_features
from manage_state import set_state, set_stand_state, state_reconciler
from algorithm_tests import trial, trial_standup, test_model, test_model_stand
from utilities import (convert_to_words, print_full, get_position_stats, combine_csv, resolve_acc_gyro,
                       blank_filter, concat_data, update_df)

TIME_SEQUENCE_LENGTH = 50

In [3]:
# create training data. We use the default time sequence length (40 rows, i.e. 1.6 seconds)
training_data = prep(40)

rolltec_features.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new_df['max_min_x'] = df2.apply(lambda x: max_min_diff(x['rolling_max_x'], x['rolling_min_x']), axis=1)
rolltec_features.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new_df['max_min_y'] = df2.apply(lambda x: max_min_diff(x['rolling_max_y'], x['rolling_min_y']), axis=1)
rolltec_features.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the ca

Removed 612 NaN rows


In [4]:
print training_data
# check the states have been created - you should see an array of values 0-7
pd.unique(training_data['state'].values.ravel())

      index     tiltx     tilty  stand   ACCEL_X   ACCEL_Y   ACCEL_Z  \
0        19 -0.260666  0.357542      0 -0.292316  0.012026 -0.900921   
1        39 -0.187004  0.056524      0 -0.279947  0.109053 -0.940895   
2        59 -0.190364  0.252252      0 -0.258079  0.114000 -0.950605   
3        79 -0.275254  0.042607      0 -0.188289  0.090132 -0.957342   
4        99  0.442236  0.669933      0 -0.073526  0.202816 -0.934947   
5       119 -0.253330  0.365558      0 -0.019816  0.199316 -0.978711   
6       139  0.044722  0.086329      0  0.115684  0.229842 -0.964263   
7       159 -0.468972  0.128312      0 -0.044368  0.179132 -0.955737   
8       179 -0.196384 -0.486311      0 -0.345605  0.081447 -0.913289   
9       199 -0.611937  0.220517      0 -0.356632  0.248789 -0.826474   
10      219 -0.348618 -0.175220      0 -0.071263  0.255737 -0.922553   
11      239 -0.346450  0.220219      0 -0.096158  0.272947 -0.908763   
12      259 -0.360857 -0.156293      0 -0.300395  0.352026 -0.85

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [5]:
# Test the accuracy of our classifier

test_model(training_data)

Random Forest prediction accuracy this time: 0.841095890411
Random Forest general accuracy: 0.73 (+/- 0.22)
Feature importance ranking:
1. feature ACCEL_Z (0.069814)
2. feature rolling_sum_z (0.067763)
3. feature rolling_max_z (0.060281)
4. feature rolling_median_z (0.056771)
5. feature rolling_min_z (0.053563)
6. feature rolling_sum_x (0.037635)
7. feature ACCEL_X (0.036748)
8. feature avg_tiltx (0.035571)
9. feature rolling_max_x (0.032326)
10. feature rolling_min_x (0.031000)
11. feature rolling_median_x (0.028793)
12. feature acc_magnitude (0.027776)
13. feature rolling_std_x (0.024480)
14. feature acc_rss (0.023605)
15. feature max_min_x (0.022956)
16. feature acc_rms (0.022523)
17. feature stand (0.021835)
18. feature rolling_std_y (0.021281)
19. feature tiltx (0.021004)
20. feature avg_stand (0.019918)
21. feature max_min_y (0.016433)
22. feature ACCEL_Y (0.015819)
23. feature max_min_z (0.015417)
24. feature rolling_median_y (0.014837)
25. feature avg_tilty (0.014569)
26. featu

In [4]:
# create some test data from 3 different test users

test_data_user1 = prep_test('GL_TEST1_CS.csv')
test_data_user2 = prep_test('GL_TEST1_UrsWearing.csv')
test_data_user3 = prep_test('DIO_OCG_YCG_YMOUNT_YBC.csv')


Removed 2 NaN rows
Removed 1 NaN rows
Removed 1 NaN rows


In [12]:
"""
This is for testing the effectiveness of the 
standing motion detection. The datasets below with STAND in the
name refer to sequences involving a stand up.

This is effectively a tangent to the main algorithm, but one which could
be very useful and widely applicable.
"""

test_data_stand = prep_test('CS_OCG_STAND_OCG.csv')

# TODO: DEBUG If I move the test data creation out of this cell it causes an error

 = trial_standup(training_data, test_data_stand)

# NOTE: BE CAREFUL WITH CREATING COPIES OF DF
step2 = trial(training_data, foo)
print step2[['state', 'avg_stand']]

# step 3

fixed = state_reconciler(step2)

test_data1000 = prep_test('URS_OCG_STAND_OCG3.csv')
baz = trial_standup(training_data40, test_data1000)
step2again = trial(training_data40, baz)
pre_pre_smooth = step2again['state'].values
pre_pre_smooth_words = convert_to_words(pre_pre_smooth)

pre_smooth = fixed['state'].values
pre_smooth_words = convert_to_words(pre_smooth)

print pre_pre_smooth_words
print pre_smooth_words

Removed 1 NaN rows
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
['your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_side_control', 'your_mount', 'your_side_control', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount', 'your_mount']
Your Mount: 0.947368421053
Your Side Control: 0.0526315789474
Your Closed Guard: 0.0
Your Back Control: 0.0
Opponent Mount or Opponent Side Control: 0.0
Opponent Closed Guard: 0.0
Opponent Back Control: 0.0
OTHER: 0.0

[7 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7 5 5 5 5 5 5 5 5 5 7 7 

NameError: global name 'update_df' is not defined

In [ ]:
n_components = 8 # ('ybc', 'ymount', 'ysc', 'ycg', 'ocg', 'osc_mount', 'obc', 'other')
startprob = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.65,]) # assume users will turn on sensor whist standing


In [15]:
"""
probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

transition_probability = {
        'ymt' : {'ymount': 0.800, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.800, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.800, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.001, 'osc_mount': 0.800, 'ocg': 0.050, 'obc': 0.050, 'other': 0.038},
        'ocg' : {'ymount': 0.100, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.800, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.010, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.800, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

transmat = np.array([
                    [0.800, 0.050, 0.010, 0.050, 0.001, 0.050, 0.001, 0.038], 
                    [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                    [0.010, 0.050, 0.800, 0.010, 0.050, 0.001, 0.001, 0.078], 
                    [0.050, 0.010, 0.050, 0.800, 0.001, 0.010, 0.001, 0.078],
                    [0.001, 0.050, 0.010, 0.001, 0.800, 0.050, 0.050, 0.038],
                    [0.100, 0.050, 0.010, 0.010, 0.001, 0.800, 0.001, 0.028],
                    [0.010, 0.050, 0.001, 0.010, 0.050, 0.001, 0.800, 0.078],
                    [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                    ])

In [40]:
"""

probability of these positions given current state:

your_mount' if v == 0 
else 'your_side_control' if v == 1
else 'your_closed_guard' if v == 2
else 'your_back_control' if v == 3
else 'opponent_mount_or_sc' if v == 4
else 'opponent_closed_guard' if v == 5
else 'opponent_back_control' if v == 6
else 'OTHER' if v == 7

emission_probability = {
        'ymt' : {'ymount': 0.500, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.050, 'osc_mount': 0.001, 'ocg': 0.350, 'obc': 0.001, 'other': 0.038},
        'ysc' : {'ymount': 0.100, 'ysc': 0.800, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.050, 'obc': 0.001, 'other': 0.028},
        'ycg' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.400, 'ybc': 0.010, 'osc_mount': 0.500, 'ocg': 0.001, 'obc': 0.001, 'other': 0.078},
        'ybc' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.600, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.201, 'other': 0.078},
        'omt' : {'ymount': 0.001, 'ysc': 0.050, 'ycg': 0.210, 'ybc': 0.050, 'osc_mount': 0.600, 'ocg': 0.050, 'obc': 0.001, 'other': 0.038},
        'ocg' : {'ymount': 0.400, 'ysc': 0.050, 'ycg': 0.010, 'ybc': 0.010, 'osc_mount': 0.001, 'ocg': 0.400, 'obc': 0.001, 'other': 0.028},
        'obc' : {'ymount': 0.010, 'ysc': 0.050, 'ycg': 0.001, 'ybc': 0.110, 'osc_mount': 0.050, 'ocg': 0.001, 'obc': 0.700, 'other': 0.078},
        'oth' : {'ymount': 0.050, 'ysc': 0.010, 'ycg': 0.050, 'ybc': 0.078, 'osc_mount': 0.001, 'ocg': 0.010, 'obc': 0.001, 'other': 0.800}
     }
"""

emissionprob = np.array([
                        [0.500, 0.050, 0.010, 0.050, 0.001, 0.350, 0.001, 0.038], 
                        [0.100, 0.800, 0.010, 0.010, 0.001, 0.050, 0.001, 0.028], 
                        [0.010, 0.050, 0.350, 0.010, 0.500, 0.001, 0.001, 0.078], 
                        [0.050, 0.010, 0.050, 0.700, 0.001, 0.010, 0.101, 0.078],
                        [0.001, 0.050, 0.210, 0.050, 0.600, 0.050, 0.001, 0.038],
                        [0.400, 0.050, 0.010, 0.010, 0.001, 0.400, 0.001, 0.028],
                        [0.010, 0.050, 0.001, 0.110, 0.050, 0.001, 0.700, 0.078],
                        [0.050, 0.010, 0.050, 0.078, 0.001, 0.010, 0.001, 0.800],
                        ])


In [41]:
# Hidden Markov Model with multinomial (discrete) emissions
model = hmm.MultinomialHMM(n_components=n_components,
                           n_iter=10,
                           verbose=False)

model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob
# model.n_features = 8

In [42]:
observations = np.array(pre_smooth)

n_samples = len(observations)
data = observations.reshape((n_samples, -1))
print data

[[5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [7]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [5]
 [0]
 [0]
 [5]
 [5]
 [7]
 [7]
 [7]]


In [43]:
print 'TEST 1'

result = model.decode(data, algorithm='viterbi')
print 'pre smooth: {}'.format(pre_pre_smooth)
print 'result accuracy {}%'.format(result[0])
print 'final result: {}'.format(result[1])

result_words = convert_to_words(result[1])
print '====================='
print 'pre pre smooth words: {}'.format(pre_pre_smooth_words)
print '====================='
print 'result words: {}'.format(result_words)


print '\n'
print "pre pre smooth stats (before stand up detection)"
print get_position_stats(pre_pre_smooth_words)


print '\n'
print "pre smooth stats (before HMM)"
print get_position_stats(pre_smooth_words)

print '\n'

print 'result stats'
print get_position_stats(result_words)

print '******************'




TEST 1
pre smooth: [5 5 5 0 0 5 5 5 5 5 5 7 5 5 5 5 5 0 5 0 0 5 5 7 7 7]
result accuracy -36.3301057979%
final result: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7 7]
pre pre smooth words: ['opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'your_mount', 'opponent_closed_guard', 'your_mount', 'your_mount', 'opponent_closed_guard', 'opponent_closed_guard', 'OTHER', 'OTHER', 'OTHER']
result words: ['opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponent_closed_guard', 'opponen